# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 19/Set até às 23:59.<br />
Grupo: 2 ou 3 pessoas - grupos com 3 pessoas terá uma rubrica diferenciada.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO gravar a key do professor no arquivo**


### Entrega Intermediária: Check 1 - APS 2

Até o dia 10/Set às 23:59, xlsx deve estar no Github com as seguintes evidências: 

  * Produto escolhido.
  * Arquivo Excel contendo a base de treinamento e a base de testes já classificadas.

Sugestão de leitura:<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Parte I - Adquirindo a Base de Dados

Acessar o notebook **Projeto-2-Planilha** para realizar a coleta dos dados. O grupo deve classificar os dados coletados manualmente.

___
## Parte II - Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Não se esqueça de implementar o Laplace Smoothing (https://en.wikipedia.org/wiki/Laplace_smoothing).

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.

Escreva o seu código abaixo:

In [25]:
#P(Relevante| "Saiu novo iphone") = P('saiu|Relevante) X P('novo'|Relevante) X P('iphone'|Relevante)

#P(palavra|Relevante) = P(Relevante)*P(palavra)

# A probabilidade de 'Relevante' não é preciso calcular

# A probabilidade de 'palavra' é calculada contando quantas vezes a 'palavra' aparece em 'Relevante' e divide pelo total de palavras

In [28]:
%matplotlib inline
import pandas as pd
import numpy as np
from math import *
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

tabela = pd.read_excel('tweets_whey.xlsx')
r_ou_i = tabela.Checando.value_counts(True)
treino = tabela.Treinamento
x = tabela.Checando
r_ou_i

0    0.743333
1    0.256667
Name: Checando, dtype: float64

In [31]:
# CONTAGEM DE PALAVRAS NO DICIONÁRIO INTEIRO

tabela.head(2)
a = []
frases = tabela.loc[:,'Treinamento']
dicio = {}
chaves = []
for i in frases:
    divide = i.split()
    a.append(divide)

conta = Counter(x for sublista in a for x in sublista)
for i in conta:
    dicio[i]=conta[i]
remover = [',', '!','.','@','/','?']
for i in remover:
    if i in dicio:
        dicio.pop(i)
for chave in dicio.keys():
    chaves.append(chave)
keys = set(chaves)
contagem_dicio = len(keys)

In [30]:
# CONTAGEM DE PALAVRAS QUANDO A FRASE É RELEVANTE

I = 0.743333
R = 0.256667

relevante = tabela[tabela.Checando==1]

a = []
frases_relevantes = relevante.loc[:,'Treinamento']
d_relevante = {}
for i in frases_relevantes:
    divide = i.split()
    a.append(divide)

conta = Counter(x for sublista in a for x in sublista)
for i in conta:
    d_relevante[i]=conta[i]
remover = [',', '!','.','@','/','?']
for i in remover:
    if i in d_relevante:
        d_relevante.pop(i)
contagem_d_relevante = pd.Series(d_relevante).sum()
contagem_d_relevante

1016

In [32]:
# CONTAGEM DE PALAVRAS QUANDO A FRASE É IRRELEVANTE

irrelevante = tabela[tabela.Checando==0]

a = []
frases_irrelevantes = irrelevante.loc[:,'Treinamento']
d_irrelevante = {}
for i in frases_irrelevantes:
    divide = i.split()
    a.append(divide)

conta = Counter(x for sublista in a for x in sublista)
for i in conta:
    d_irrelevante[i]=conta[i]
remover = [',', '!','.','@','/','?']
for i in remover:
    if i in d_irrelevante:
        d_irrelevante.pop(i)
contagem_d_irrelevante = pd.Series(d_irrelevante).sum()
contagem_d_irrelevante

4086

In [33]:
#Classificador

def Classificador(Lista):
    Lista_classificada = []
    for i in Lista:
        w= i.split()
        PR = []
        PNR = []
        for k in w:
            PR.append(Prob_Rr(k))
            PNR.append(Prob_NRr(k))
        P_Relevante = 1
        P_Irrelevante = 1
        for i in PR:
            P_Relevante *= i
        for i in PNR:
            P_Irrelevante *= i
        Prel = P_Relevante*Prob_R
        Pirrel = P_Irrelevante*Prob_NR
        if Prel > Pirrel:
            Lista_classificada.append('Relevante')
        else:
            Lista_classificada.append('Irrelevante')
    return Lista_classificada

#prob_relevante = {}
#prob_irrelevante = {}
#for palavra in dicio:
   # if palavra in d_relevante:
    #    conta_relevante= (d_relevante[palavra]+1)/(contagem_d_relevante+contagem_dicio)
      #  prob_relevante[palavra] = conta_relevante
    
   # if palavra in d_irrelevante:
    #    conta_irrelevante = (d_irrelevante[palavra]+1)/(contagem_d_irrelevante+contagem_dicio)
    #    prob_irrelevante[palavra] = conta_irrelevante
#prob_relevante['bom']    
    

0.0011320754716981133

In [34]:
lista = []
for i in tabela.Treinamento:
    frase = i
    divide = frase.split()
    positivo = 1
    negativo = 1
    for palavra in divide:
        if palavra in d_relevante:
            positivo *= prob_relevante[palavra]
        if palavra in d_irrelevante:
            negativo *= prob_irrelevante[palavra]    
        if palavra not in d_relevante:
            conta_relevante_zero= 1/(contagem_d_relevante+contagem_dicio)
            prob_relevante[palavra] = conta_relevante_zero   
            positivo *= prob_relevante[palavra]
        if palavra not in d_irrelevante:
            conta_irrelevante_zero = 1/(contagem_d_irrelevante+contagem_dicio)
            prob_irrelevante[palavra] = conta_irrelevante_zero
            negativo *= prob_irrelevante[palavra]    
        #print('REl {0}\nIRRel {1}'.format(prob_relevante[palavra],prob_irrelevante[palavra]))
    positivo_final = positivo*R
    negativo_final = negativo*I
    #print('REl {0}\nIRREl {1}'.format(positivo_final,negativo_final))
    if positivo_final > negativo_final:
        lista.append(1)
    else:
        lista.append(0)
tabela['Computador'] = lista
tabela.head(2)

,Treinamento,Checando,Computador
0,rt @galaumironic: ô juiz se você acabar o jogo...,0,0
1,mano whey é mt ruim quem q gosta de tomar isso,1,1


___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Obrigatório para grupos de 3 alunos:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [35]:
teste = pd.read_excel('tweets_whey.xlsx','Teste')
teste.Teste[:2]

0    @diegolugano é so contratar um goleiro !!!\n\n...
1    comprei um shake de whey protein q tem gosto d...
Name: Teste, dtype: object

In [36]:
lista1 = []
for i in teste:
    frase = i
    divide = frase.split()
    positivo = 1
    negativo = 1
    for palavra in divide:
        if palavra in d_relevante:
            positivo *= prob_relevante[palavra]
        if palavra in d_irrelevante:
            negativo *= prob_irrelevante[palavra]    
        if palavra not in d_relevante:
            conta_relevante_zero= 1/(contagem_d_relevante+contagem_dicio)
            prob_relevante[palavra] = conta_relevante_zero   
            positivo *= prob_relevante[palavra]
        if palavra not in d_irrelevante:
            conta_irrelevante_zero = 1/(contagem_d_irrelevante+contagem_dicio)
            prob_irrelevante[palavra] = conta_irrelevante_zero
            negativo *= prob_irrelevante[palavra]    
        #print('REl {0}\nIRRel {1}'.format(prob_relevante[palavra],prob_irrelevante[palavra]))
    positivo_final = positivo*R
    negativo_final = negativo*I
    #print('REl {0}\nIRREl {1}'.format(positivo_final,negativo_final))
    if positivo_final > negativo_final:
        lista.append(1)
    else:
        lista.append(0)
teste['Comparador'] = lista1
teste.head(2)

ValueError: Length of values does not match length of index

In [37]:
Frases_teste = []
for i in teste:
    Frases_teste.append(i)
Teste_Final = Classificador(Frases_teste)
print(Teste_Final)

NameError: name 'Classificador' is not defined

In [ ]:
#teste = Data_Teste.Relevância

PositivosF= 0
for i in range(len(Teste_Final)) and range(len(teste)):
    if Final[i] == 'Relevante' and teste[i] == "0":
        Positivos_Falsos += 1
print('Positivos Falsos: {0}'.format(Positivos_Falsos/200))

Positivos_Verdadeiros = 0
for i in range(len(Teste_Final)) and range(len(teste)):
    if Final[i] == 'Relevante' and teste[i] == "1":
        Positivos_Verdadeiros += 1
print('Positivos Verdadeiros: {0}'.format(Positivos_Verdadeiros/200))

Negativos_Verdadeiros = 0
for i in range(len(Teste_Final)) and range(len(teste)):
    if Final[i] == 'Irrelevante' and teste[i] == "0":
        Negativos_Verdadeiros += 1
print('Negativos Verdadeiros: {0}'.format(Negativos_Verdadeiros/200))

Negativos_Falsos = 0
for i in range(len(Teste_Final)) and range(len(teste)):
    if Final[i] == 'Irrelevante' and teste[i] == "1":
        Negativos_Falsos += 1
print('Negativos Falsos: {0}'.format(Negativos_Falsos/200))

___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).
